# TensorFlow Tutorial

2015년 12월 오픈소스로 공개된 TensorFlow, 속칭 tf는 Google의 주도로 개발된 기계학습(딥러닝) 라이브러리이다. Multi-GPU 환경에서도 동작가능한 아키텍처 기반의 TensorFlow는 Theano의 Computation Graph 방법을 채택하여 automatic gradient differentiation이 가능하다.

TensorFlow는 CPU모드, GPU, 그리고 Multi-GPU 환경에서 모두 사용가능하며, 직관적이고 쉬운 문법으로 인한 높은 접근성, 구글의 강력한 지원하에 이루어지는 빠른 업데이트, 그리고 큰범위의 개발자생태계가 존재하여 쉽게 Trouble Shooting이 가능하다는 큰 장점을 갖고있다.


#### 관련 사이트
TensorFlow 공식 홈페이지 : http://tensorflow.org/
<br>TensorFlow Github : https://github.com/tensorflow/tensorflow/releases
<br>TensorFlow Contributors : https://github.com/tensorflow/tensorflow/graphs/contributors
<br>TensorFlow Playground : http://playground.tensorflow.org/

<img src = "computation graph.png" width=600 >
https://delftswa.gitbooks.io/desosa2016/content/tensorflow/chapter.html

### TensorFlow의 핵심 원리
TensorFlow의 중요한 핵심 개념인 Computation Graph와 Session에 대해 알아보자.
Computation Graph란 tf에서 실행되는 모든 알고리즘 혹은 모델을 일련의 node와 edge들의 연결로 하나의 graph 로 나타내는 것이라고 할 수 있다. 

Computation Graph 기반의 프레임워크가 가진 장점은 모델 구현의 유연성이 크고, 자동화된 미분(Automatic Differentiation) 계산이 가능하다는 점이 있다.

Session이란 한마디로 Computation Graph가 실행되는 환경 혹은 자원을 의미한다. 일반적으로 딥러닝 연산에는 GPU를 활용하게 되는데, 여기에서 확보가능한 메모리 및 프로세서 자원을 추상화시켜 Session이라고 표현할 수 있다.

정리하면 Computation Graph는 알고리즘 또는 딥러닝 모델, Session은 모델이 실행되는 GPU환경 이라고 할 수 있다.

그러면 tf의 기본적인 변수 선언 방법 및 연산 사용 방법을 알아보자.

In [1]:
import tensorflow as tf

#configuration setting for gpu usage control
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Simple matrix multiplication

행렬을 정의하고 행렬곱을 수행하는 Computation Graph를 tensorflow로 구성한다.
tf.Session로 구성하고 sess.run으로 작동시키면 병렬적으로 모든 결과값을 처리 후 결과를 반환한다.

In [2]:
mat1 = tf.constant([[3., 3.], [4.,4.]]) # 2x2 matrix
mat2 = tf.constant([[2.,],[2.]])#2x1 matrix
 
product = tf.matmul(mat1, mat2)

#1
sess = tf.Session(config=config)
 
result = sess.run(product) # to get result
print(result)
 
sess.close()

[[ 12.]
 [ 16.]]


# InteractiveSeesion

tf.InteractiveSession을 사용하게 될 경우 기존 tf.Session 에서 매번 오브젝트를 참조할 수고를 덜어준다.

ex)
<br>**Session 사용 시**  ## 깔끔한 코딩을 위해 session을 사용하자. 
<pre>
t = tf.constant(42.0)
sess = tf.Session()
with sess.as_default():   # or `with sess:` to close on exit
    assert sess is tf.get_default_session()
    assert t.eval() == sess.run(t)
</pre>

**InteractiveSession 사용 시**
<pre>
sess = tf.InteractiveSession()
t = tf.constant(42.0)
u = tf.constant(37.0)
tu = tf.mul(t, u)
ut = tf.mul(u, t)
with sess.as_default():
    tu.eval() # runs one step
</pre>

쉽게 생각하여 예를들면 for와 while 문과 같은 관계(?)라고 어느정도 예상 할 수 있다. 
<br>(reference) http://stackoverflow.com/questions/33610685/in-tensorflow-what-is-the-difference-between-session-run-and-tensor-eval

In [3]:
sess = tf.InteractiveSession("",config=config)
 
x = tf.Variable([1.0, 2.0]) #variable need initialization
a = tf.constant([3.0, 3.0]) #constant no initiazliation

#1 
x.initializer.run()
 
add_up = tf.add(x, a)
result = add_up.eval() # to get result
print(result)

sess.close()


[ 4.  5.]


# Using variables

값을 저장할 수 있는 컨테이너 역할을 하는 Variable.
<br>하지만 사용하기 위해서는 initialize가 필요하다.
<br>아래 예제에서 counter variable이 run(update)를 할때마다 1씩 증가하는것을 볼 수 있다.

In [4]:
counter = tf.Variable(0, name="counter") # tf.Variable(<initial-value>, name=<optional-name>)

one = tf.constant(1) # constant 1
upcount = tf.add(counter, one) # add
update = tf.assign(counter, upcount) # counter = upcount 
 
init_op = tf.initialize_all_variables() #initializer
 
with tf.Session(config=config) as sess: #start with tf.Session()
    sess.run(init_op) #run (init_op)
    print(sess.run(counter)) #run (state)
    for _ in range(3):
        sess.run(update) #run(update)
        print(sess.run(counter)) # print after update

sess.close()    

0
1
2
3


# Using feed

tf.placeholder는 우리가 원하는 데이터를 Computation Graph에 입력시켜주는 역할을 하는 변수이다. 즉 input 을 받기 위한 변수라고 생각할 수 있다.

Session이 수행될 때 해당 placeholder에 값을 입력해주는 방법으로 feed_dict 방법이 있다.

아래 예제에서는 input1과 input2에 값을 입력해주고 output을 fetch한다.

In [5]:
input1 = tf.placeholder(tf.float32) # recieve 1x1 matrix with type float32
input2 = tf.placeholder(tf.float32) # recieve 1x1 matrix with type float32
output = tf.add(input1, input2)
 
with tf.Session(config=config) as sess:
    print "lucky"
    #1
    print(sess.run([output], feed_dict={input1:[5.], input2:[2.]}))

sess.close()

lucky
[array([ 7.], dtype=float32)]
